In [19]:
# %pip install openpyxl et_xmlfile --upgrade

In [20]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
from datetime import datetime
import openpyxl

In [24]:
idw_path = r'.\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf'
cn_path_station = r'.\1 - Organized data gauge\NormaisClimatologicas\Estacoes'
cn_path_prec = r'.\1 - Organized data gauge\NormaisClimatologicas\Prec'

In [22]:
def convert_degrees(string):
    parts = string.split('°')
    degrees = float(parts[0])
    minutes = float(parts[1].strip("'").split('\'')[0])
    direction = parts[1].strip("'").split('\'')[1]
    
    decimal_degrees = degrees + minutes / 60
    
    if direction == 'S' or direction == 'W':
        decimal_degrees *= -1
    
    return decimal_degrees

In [23]:
def pr_value_calc(lat, lon, df):
    df_temp = df.sel(long = lon, lat = lat, method = 'nearest')
    return df_temp.variables['rain_mm'].to_numpy()

In [28]:
all_files = glob.glob(idw_path + r"\*.h5")
all_files.sort()
total_length = len(all_files)
print(total_length)

23376


In [29]:
all_files[0] #find first date

'.\\1 - Organized data gauge\\BRAZIL\\NetCDF\\IDW_optimization_hdf\\precipitation_idw_1961-01-01.h5'

In [30]:
all_files[10957] #find last date

'.\\1 - Organized data gauge\\BRAZIL\\NetCDF\\IDW_optimization_hdf\\precipitation_idw_1991-01-01.h5'

In [31]:
all_files_ref = glob.glob(cn_path_station + r"\*.xlsx")
all_files_ref

['.\\1 - Organized data gauge\\NormaisClimatologicas\\Estacoes\\NC_ESTACOES_1961_1990.xlsx',
 '.\\1 - Organized data gauge\\NormaisClimatologicas\\Estacoes\\NC_ESTACOES_1981_2010.xlsx',
 '.\\1 - Organized data gauge\\NormaisClimatologicas\\Estacoes\\NC_ESTACOES_1991_2020.xlsx']

In [32]:
df_cn_station = pd.read_excel(r'.\\1 - Organized data gauge\\NormaisClimatologicas\\Estacoes\\NC_ESTACOES_1961_1990.xlsx')
df_cn_station = df_cn_station[["Código","Nome",		"UF",	"Latitude",	"Longitude",	"Altitude (m)"]]
df_cn_station['Latitude'] = df_cn_station['Latitude'].apply(convert_degrees)
df_cn_station['Longitude'] = df_cn_station['Longitude'].apply(convert_degrees)
df_cn_station.columns = ['Código', 'Nome', 'UF', 'Latitude_Decimal', 'Longitude_Decimal', 'Altitude (m)']
df_cn_station

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m)
0,83010,Brasiléia,AC,-11.016667,-68.733333,260.00
1,82704,Cruzeiro do Sul,AC,-7.633333,-72.666667,170.00
2,82915,Rio Branco,AC,-9.966667,-67.800000,160.00
3,82807,Tarauacá,AC,-8.166667,-70.766667,190.00
4,83098,Coruripe,AL,-10.150000,-36.166667,56.13
...,...,...,...,...,...,...
389,83231,Paranã,TO,-12.550000,-47.833333,275.00
390,82863,Pedro Afonso,TO,-8.966667,-48.183333,187.00
391,83228,Peixe,TO,-12.016667,-48.350000,242.49
392,83064,Porto Nacional,TO,-10.716667,-48.416667,239.20


In [40]:
count = 0
total_len = 0
df_result = []
for filename in all_files[0:10957]:
    print(count, filename)
    count = count + 1
    date_str = filename[-13:-3]
    print(date_str)
    ref_date = datetime.strptime(date_str, "%Y-%m-%d")
    df_temp = df_cn_station.copy(deep=True)
    lat_list = df_temp['Latitude_Decimal'].values.tolist()
    lon_list = df_temp['Longitude_Decimal'].values.tolist()
    coord_list = []
    for i in range(len(lat_list)):
        # print(lat_list[i])
        coord_list.append([lat_list[i], lon_list[i]])
    df = pd.read_hdf(filename, mode='r')
    ds = xr.Dataset.from_dataframe(df.set_index(['lat', 'long']))    
    pr_list = []
    for coord in coord_list:
        pr_value = pr_value_calc(coord[0], coord[1], ds)
        pr_list.append(pr_value)
        # print(pr_value)
    # print(pr_list)
    df_temp['interpolated_rain_mm'] = pr_list
    df_temp['Date'] = ref_date
    # print(df_temp.head(5))
    if len(df_result) == 0:
        df_result = df_temp.copy(deep=True)
    else:
        df_result = pd.concat([df_result, df_temp], ignore_index= True)
    total_len = total_len + len(df_temp)
df_result

0 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-01.h5
1961-01-01
1 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-02.h5
1961-01-02
2 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-03.h5
1961-01-03
3 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-04.h5
1961-01-04
4 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-05.h5
1961-01-05
5 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-06.h5
1961-01-06
6 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-07.h5
1961-01-07
7 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-08.h5
1961-01-08
8 .\1 - Organized data gauge\BRAZIL\NetCDF\IDW_optimization_hdf\precipitation_idw_1961-01-09.h5
1961-01-09
9 .\1 - Organized data gauge\BRAZIL\N

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m),interpolated_rain_mm,Date
0,83010,Brasiléia,AC,-11.016667,-68.733333,260.00,12.2057178585898,1961-01-01
1,82704,Cruzeiro do Sul,AC,-7.633333,-72.666667,170.00,10.505241508857656,1961-01-01
2,82915,Rio Branco,AC,-9.966667,-67.800000,160.00,11.911291574961345,1961-01-01
3,82807,Tarauacá,AC,-8.166667,-70.766667,190.00,10.799464915708022,1961-01-01
4,83098,Coruripe,AL,-10.150000,-36.166667,56.13,0.0,1961-01-01
...,...,...,...,...,...,...,...,...
4317053,83231,Paranã,TO,-12.550000,-47.833333,275.00,0.12900213264237798,1990-12-31
4317054,82863,Pedro Afonso,TO,-8.966667,-48.183333,187.00,0.14131117917411848,1990-12-31
4317055,83228,Peixe,TO,-12.016667,-48.350000,242.49,6.9270274568079095,1990-12-31
4317056,83064,Porto Nacional,TO,-10.716667,-48.416667,239.20,0.32914770650017827,1990-12-31


In [42]:
df_result['Date'] = pd.to_datetime(df_result['Date'])
df_result['Year'] = df_result['Date'].dt.year
df_result['Month'] = df_result['Date'].dt.month
df_result['Date'] = df_result['Date'].astype(str)
df_result 

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m),interpolated_rain_mm,Date,Year,Month
0,83010,Brasiléia,AC,-11.016667,-68.733333,260.00,12.2057178585898,1961-01-01,1961,1
1,82704,Cruzeiro do Sul,AC,-7.633333,-72.666667,170.00,10.505241508857656,1961-01-01,1961,1
2,82915,Rio Branco,AC,-9.966667,-67.800000,160.00,11.911291574961345,1961-01-01,1961,1
3,82807,Tarauacá,AC,-8.166667,-70.766667,190.00,10.799464915708022,1961-01-01,1961,1
4,83098,Coruripe,AL,-10.150000,-36.166667,56.13,0.0,1961-01-01,1961,1
...,...,...,...,...,...,...,...,...,...,...
4317053,83231,Paranã,TO,-12.550000,-47.833333,275.00,0.12900213264237798,1990-12-31,1990,12
4317054,82863,Pedro Afonso,TO,-8.966667,-48.183333,187.00,0.14131117917411848,1990-12-31,1990,12
4317055,83228,Peixe,TO,-12.016667,-48.350000,242.49,6.9270274568079095,1990-12-31,1990,12
4317056,83064,Porto Nacional,TO,-10.716667,-48.416667,239.20,0.32914770650017827,1990-12-31,1990,12


In [43]:
all_files = glob.glob(cn_path_prec + r"\*")
all_files

['.\\1 - Organized data gauge\\NormaisClimatologicas\\Prec\\BRASIL_CLIMATOLOGICAL_NORMALS.h5',
 '.\\1 - Organized data gauge\\NormaisClimatologicas\\Prec\\NC_PREC_1961_1990.xlsx',
 '.\\1 - Organized data gauge\\NormaisClimatologicas\\Prec\\NC_PREC_1981_2010.xlsx',
 '.\\1 - Organized data gauge\\NormaisClimatologicas\\Prec\\NC_PREC_1991_2020.xlsx']

In [44]:
df_result.to_hdf(cn_path_prec+"\\BRASIL_CLIMATOLOGICAL_NORMALS.h5", key = 'table_data_1961_1990', mode = 'r+', append = False, complevel = 9, encoding="utf-8")
# df_interpolation = pd.read_hdf(cn_path_prec + r'\BRASIL_CLIMATOLOGICAL_NORMALS.h5', KEY='table_data_1961_1990')
# df_interpolation

C:\Users\linde\AppData\Local\Temp\ipykernel_22708\1709972572.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['Nome', 'UF', 'interpolated_rain_mm', 'Date'], dtype='object')]

  df_result.to_hdf(cn_path_prec+"\\BRASIL_CLIMATOLOGICAL_NORMALS.h5", key = 'table_data_1961_1990', mode = 'r+', append = False, complevel = 9, encoding="utf-8")


In [45]:
df_interpolation_v1 = pd.read_hdf(cn_path_prec + "\\BRASIL_CLIMATOLOGICAL_NORMALS.h5", key='table_data_1961_1990')
df_interpolation_v1

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m),interpolated_rain_mm,Date,Year,Month
0,83010,Brasiléia,AC,-11.016667,-68.733333,260.00,12.2057178585898,1961-01-01,1961,1
1,82704,Cruzeiro do Sul,AC,-7.633333,-72.666667,170.00,10.505241508857656,1961-01-01,1961,1
2,82915,Rio Branco,AC,-9.966667,-67.800000,160.00,11.911291574961345,1961-01-01,1961,1
3,82807,Tarauacá,AC,-8.166667,-70.766667,190.00,10.799464915708022,1961-01-01,1961,1
4,83098,Coruripe,AL,-10.150000,-36.166667,56.13,0.0,1961-01-01,1961,1
...,...,...,...,...,...,...,...,...,...,...
4317053,83231,Paranã,TO,-12.550000,-47.833333,275.00,0.12900213264237798,1990-12-31,1990,12
4317054,82863,Pedro Afonso,TO,-8.966667,-48.183333,187.00,0.14131117917411848,1990-12-31,1990,12
4317055,83228,Peixe,TO,-12.016667,-48.350000,242.49,6.9270274568079095,1990-12-31,1990,12
4317056,83064,Porto Nacional,TO,-10.716667,-48.416667,239.20,0.32914770650017827,1990-12-31,1990,12


In [46]:
df_interpolation_v2 = pd.read_hdf(cn_path_prec + "\\BRASIL_CLIMATOLOGICAL_NORMALS.h5", key='table_data_1991_2020')
df_interpolation_v2

,Código,Nome,UF,Latitude_Decimal,Longitude_Decimal,Altitude (m),interpolated_rain_mm,Date,Year,Month
0,82704,CRUZEIRO DO SUL,AC,-7.610736,-72.681324,214.19,0.023620,1991-01-01,1991,1
1,82915,RIO BRANCO,AC,-9.959167,-67.868889,160.71,8.703896,1991-01-01,1991,1
2,82807,TARAUACA,AC,-8.160278,-70.768611,172.27,0.000000,1991-01-01,1991,1
3,82989,AGUA BRANCA,AL,-9.264722,-37.938056,603.42,0.471105,1991-01-01,1991,1
4,82994,MACEIO,AL,-9.551389,-35.770833,84.12,2.051808,1991-01-01,1991,1
...,...,...,...,...,...,...,...,...,...,...
2969613,83033,PALMAS,TO,-10.190897,-48.301822,291.68,0.000000,2020-12-31,2020,12
2969614,82863,PEDRO AFONSO,TO,-8.968611,-48.177222,189.53,16.902135,2020-12-31,2020,12
2969615,83228,PEIXE,TO,-12.015278,-48.545000,252.24,0.000000,2020-12-31,2020,12
2969616,83064,PORTO NACIONAL,TO,-10.710833,-48.406389,243.28,0.000000,2020-12-31,2020,12
